In [1]:
from custom_dataset import Dataset_Assay, Dataset_Assay_Survival, custom_collate_assay, custom_collate_assay_survival
from models import *
from saveAndLoad import *

# LOAD DATA
data_dir = '../labeled_data/'
labeled_data = os.listdir(data_dir)
for ni,i in sorted(zip(labeled_data,range(len(labeled_data)))):print(i,'\t',ni)
data_emb = labeled_data[6]
print(data_emb)
data_df_emb = pd.read_csv(data_dir+data_emb)

nlabels = len(data_df_emb['int_label'].unique())
print('n labels:',nlabels)
device = 'cuda:1'

mut_embeddings = np.load('../aa/canonical_mut_average_embeddings_esm2.npy')
ref_embeddings = np.load('../aa/canonical_ref_embeddings_esm2.npy')
tumors = pickleLoad('../aa/tumors.pkl')
assays = pickleLoad('../aa/assays.pkl')

/home/dandreas/.conda/envs/deepV_a100/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/home/dandreas/.conda/envs/deepV_a100/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


0 	 BINARYdata_CANCER_TYPE_0_00percentMinCancerType.csv
12 	 BINARYdata_CANCER_TYPE_0_10percentMinCancerType.csv
2 	 BINARYdata_CANCER_TYPE_1_00percentMinCancerType.csv
4 	 BINARYdata_CANCER_TYPE_DETAILED_0_00percentMinCancerType.csv
11 	 BINARYdata_CANCER_TYPE_DETAILED_0_10percentMinCancerType.csv
9 	 BINARYdata_CANCER_TYPE_DETAILED_1_00percentMinCancerType.csv
6 	 data_CANCER_TYPE_0_00percentMinCancerType.csv
5 	 data_CANCER_TYPE_0_10percentMinCancerType.csv
10 	 data_CANCER_TYPE_1_00percentMinCancerType.csv
1 	 data_CANCER_TYPE_DETAILED_0_00percentMinCancerType.csv
3 	 data_CANCER_TYPE_DETAILED_0_10percentMinCancerType.csv
7 	 data_CANCER_TYPE_DETAILED_1_00percentMinCancerType.csv
8 	 label_mappings
data_CANCER_TYPE_0_00percentMinCancerType.csv
n labels: 108
loading data from ../aa/tumors.pkl
loading data from ../aa/assays.pkl


In [2]:
for i in [6,5,10,1,3,7]:
    print(labeled_data[i])
    df = pd.read_csv(data_dir+labeled_data[i])
    Dataset_Assay(df,mut_embeddings,ref_embeddings,tumors,assays,'cpu')
    print('')

data_CANCER_TYPE_0_00percentMinCancerType.csv
159445 samples
142745 unique patients
108 labels

data_CANCER_TYPE_0_10percentMinCancerType.csv
157887 samples
141445 unique patients
53 labels

data_CANCER_TYPE_1_00percentMinCancerType.csv
138303 samples
124702 unique patients
20 labels

data_CANCER_TYPE_DETAILED_0_00percentMinCancerType.csv
159445 samples
142745 unique patients
783 labels

data_CANCER_TYPE_DETAILED_0_10percentMinCancerType.csv
141057 samples
127155 unique patients
143 labels

data_CANCER_TYPE_DETAILED_1_00percentMinCancerType.csv
82664 samples
75447 unique patients
17 labels



In [3]:
dataset = Dataset_Assay(data_df_emb, mut_embeddings, ref_embeddings, tumors, assays, device)
train_loader, test_loader = getTrainTestLoaders(dataset, batch_size = 750, collate=custom_collate_assay)

138303 samples
124702 unique patients
20 labels


In [ ]:
## ATTENTION MODEL WITH POSITION EMBEDDING
class Config_Att:
    n_layer: int = 3
    input_dim: int = 640
    dropout: float = 0.0
    bias: bool = False
    n_labels: int
    pooling : str = 'mean'
    norm_fn: nn.Module = nn.LayerNorm
    max_len : int = 1448
    position_embedding: bool = False
    num_heads: int = 1

config_att = Config_Att()
config_att.n_labels = nlabels

model = Classifier(config_att)
model.to(device)

num_epochs = 15
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.001)
train_assay(model,num_epochs,train_loader,test_loader, criterion, optimizer)

In [4]:
dataset = Dataset_Assay_Survival(data_df_emb, mut_embeddings, ref_embeddings, tumors, assays, device, ratio=1)
train_loader, test_loader = getTrainTestLoaders(dataset, batch_size = 500, collate=custom_collate_assay_survival)

dropped 17595 nan samples (11.04%)
41026 samples
39291 unique patients


In [6]:
## ATTENTION MODEL WITH POSITION EMBEDDING
class Config_Att:
    n_layer: int = 3
    input_dim: int = 640
    dropout: float = 0.0
    bias: bool = False
    n_labels: int = 1
    pooling : str = 'mean'
    norm_fn: nn.Module = nn.LayerNorm
    max_len : int = 1448
    position_embedding: bool = False
    num_heads: int = 1

config_att = Config_Att()

model = Classifier(config_att)
model.to(device)

num_epochs = 5
optimizer = optim.Adam(model.parameters(), lr=.00001)
criterion = negative_log_partial_likelihood
# train_assay_survival(model,num_epochs,train_loader,test_loader, criterion, optimizer, saveName = './best_models/model_sha_survival_uncensored.pt')
train_assay_survival(model,num_epochs,train_loader,test_loader, criterion, optimizer)

TESTING: 100%|██████████| 17/17 [00:21<00:00,  1.29s/it]


C-Index: 0.7578


TESTING: 100%|██████████| 17/17 [00:21<00:00,  1.29s/it]


C-Index: 0.7645


TESTING: 100%|██████████| 17/17 [00:30<00:00,  1.82s/it]


C-Index: 0.7679


TESTING: 100%|██████████| 17/17 [00:22<00:00,  1.33s/it]


C-Index: 0.7685


TESTING: 100%|██████████| 17/17 [00:19<00:00,  1.15s/it]

C-Index: 0.7673
Best C-Index: 0.7685 at epoch 3
